<a href="https://colab.research.google.com/github/BakuDoriya-KatsZuku/Whatsapp_Chat_Reports/blob/main/WhatsApp_Chat_Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import datetime
from textblob import TextBlob
import plotly.graph_objs as go
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

with open("/content/WhatsApp Chat with The Alt Cult 🚩🚩.txt") as f:
    lines = f.readlines()

dates, times, names, messages = [], [], [], []

for line in lines:
    line.strip("\n")
    if " - " in line:
        parts = line.split(" - ")
        if len(parts) == 2:
            date_time = parts[0].strip().split(', ')
            if len(date_time) == 2:
                date, time = date_time
                x = parts[1].split(":")
                name = ''
                if len(x) == 2:
                  name, message = x
                  dates.append(date)
                  times.append(time)
                  names.append(name)
                  messages.append(message)

df = pd.DataFrame({
    "Date": dates,
    "Time": times,
    "Name": names,
    "Message": messages
})

df["Message"] = df["Message"].str.strip("\n")

In [2]:
x = dict(df["Name"].value_counts())
x = pd.DataFrame(x.items(), columns = ["Name", "Count"])
fig = px.bar(x, x = "Name", y = "Count", title = "Number of Messages")
fig.show()

In [3]:
#Make DF words and count (Keeping this so that we can also use for plotting)
x = {}
for i in range(df.shape[0]):
  words = df["Message"].iloc[i].split()
  for word in words:
    if word not in x:
      x[word] = 1
    else:
      x[word] += 1
x = pd.DataFrame(x.items(), columns = ["Word", "Count"])

#Removing Media
z = x[(x["Word"] == "<Media") | (x["Word"] == "omitted>")].index
x.drop(z, inplace = True)
x = x.sort_values(by = 'Count', ascending = False)
x = x.loc[x["Count"] >= 10]
print("Most Messaged Words")
print(x.to_string(index = False))

Most Messaged Words
           Word  Count
            hai   5827
             ka   1957
             ke   1811
              💙   1784
            bhi   1781
           nahi   1740
            toh   1642
            kya   1372
              ?   1301
             me   1173
             se   1126
             ko   1096
             na   1042
            tha   1036
             hi    979
             to    886
           raha    866
            kar    857
        message    711
            mea    683
            was    676
             ho    674
             ki    671
        deleted    663
           This    650
             ye    616
           bhai    614
            nhi    608
             pe    607
            sab    598
            aur    542
              💛    541
             wo    514
            the    510
           kuch    502
             Ha    495
           main    473
             ek    472
             Ye    471
             aa    468
           null    465
            an

In [4]:
#Making Dictionary for words with lowercase (IDK why groupby does not give result. It just prints unkown characters)
x = {}
for i in range(df.shape[0]):
  words = df["Message"].iloc[i].split()
  for word in words:
    word = word.lower()
    if word not in x:
      x[word] = 1
    else:
      x[word] += 1
x = pd.DataFrame(x.items(), columns = ["Word", "Count"])
x = x.sort_values(by = 'Count', ascending = False)

print(x.loc[x["Word"].isin(['bsdk','fuck', 'mc', 'bc', "f"])])

      Word  Count
550     bc    283
1359  bsdk     88
1318    mc     38
99       f     31
1147  fuck     22


In [5]:
new_df = df[df.Message != "<Media omitted>"] #Removing Medai ommitted so that they do not hamper word counts

z = df["Name"].unique() #List of unique names

#Dictionary to count the number of words in each name
x = {}
for name in z:
  x[name] = 0
for i in range(new_df.shape[0]):
  x[df["Name"].iloc[i]] += len(new_df["Message"].iloc[i].split())

#Sortig the Dictionary in reverse order
keys = list(x.keys())
values = list(x.values())
sorted_value_index = np.argsort(values)
x = {keys[i]: values[i] for i in sorted_value_index}
x = dict(reversed(list(x.items())))

#Plot
x = pd.DataFrame(x.items(), columns = ["Name", "Count"])
fig = px.bar(x, x = "Name", y = "Count", title = "Number of Words")
fig.show()

In [6]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True) #Forcing all the values in the column to become in that format else NAN
if df['Date'].isnull().sum() > 0:
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%d-%m', errors='coerce')
df["Date"] = df["Date"].dropna() #Drop all useless values
df['Month_Year'] = df['Date'].dt.strftime('%B %Y') #Converting to Month year for better looking on the graph
grouped = df.groupby(['Name', 'Month_Year', 'Date']).count() #Grouping these values ie making them the primary key

# Performing Operations for each and every person
for name in grouped.index.levels[0]:
    sub_df = grouped.loc[name].reset_index()
    sub_df = sub_df.sort_values(by='Date')
    #Replacing the missing date values ie where the person did not message and putting the message count to 0 over there
    sub_df = sub_df.set_index("Date")
    sub_df = sub_df.resample("D").asfreq() #Replace missing date values (Can change only if index is the column whose dates you want to insert ie all the values there have to be unique)
    sub_df["Message"].fillna(0, inplace=True)
    sub_df = sub_df.reset_index() #Change index again
    #Plot
    fig = px.line(sub_df, x = 'Date', y = 'Message', title = name)
    fig.show()

#Same but not bringing back in the dates (Can't do because they repeat because of repeating name. One way is the create different dataframes and then concat them and then plot with color = "name" but takes too much time so did not do it)
grouped = grouped.reset_index()
grouped = grouped.sort_values(by='Date')
fig = px.line(grouped, x = 'Date', y = 'Message', color = 'Name', title = 'Everyone')
fig.show()

#Same thing but for the whole group so ignoring the names
grouped = df.groupby(['Date']).count()
grouped = grouped.reset_index()
grouped = grouped.sort_values(by = 'Date')
grouped = grouped.set_index("Date")
grouped = grouped.resample("D").asfreq()
grouped["Message"].fillna(0, inplace=True)
grouped = grouped.reset_index()
fig = px.line(grouped, x = 'Date', y = 'Message', title = 'Overall messages of the group')
fig.show()

In [7]:
#Using textblob for sentimental analysis (IDK how to use it so chatGPT(Orz))
df['Sentiment'] = df['Message'].apply(lambda x: TextBlob(x).sentiment.polarity)
grouped = df.groupby(['Name'])
sentiment_scores = grouped['Sentiment'].mean() #Finding mean because just 1 number of each person look good

#Plot
fig = px.bar(sentiment_scores, x=sentiment_scores.index, y=sentiment_scores, title='Sentiment Scores by Person')
fig.show()

In [8]:
#Instead of messages did the same process for number of words
df['Word Count'] = df['Message'].apply(lambda x: len(x.split()))
grouped = df.groupby(['Date', 'Name'])
word_count_per_day = grouped['Word Count'].sum().reset_index()

for name, sub_df in word_count_per_day.groupby('Name'):
    sub_df = sub_df.sort_values(by='Date')
    sub_df = sub_df.set_index("Date")
    sub_df = sub_df.resample("D").asfreq()
    sub_df["Word Count"].fillna(0, inplace=True)
    sub_df = sub_df.reset_index()
    fig = px.line(sub_df, x='Date', y='Word Count', title=name)
    fig.show()

overall_word_count = word_count_per_day.groupby('Date').sum().reset_index()
overall_word_count = overall_word_count.sort_values(by='Date')
overall_word_count = overall_word_count.set_index("Date")
overall_word_count = overall_word_count.resample("D").asfreq()
overall_word_count["Word Count"].fillna(0, inplace=True)
overall_word_count = overall_word_count.reset_index()

fig = px.line(overall_word_count, x='Date', y='Word Count', title='Overall Word Count of the group')
fig.show()

<ipython-input-8-09645f4a6b0f>:15: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [9]:
#Instead of Date using hours as a metric to see which time of the day is most active
def convert(string):
      if string[-2:] == "am" and string[:2] == "12":
         return "00" + string[2:5]
      elif string[-2:] == "am":
         return string[:2]
      elif string[-2:] == "pm" and string[:2] == "12":
         return string[:5]
      elif string[1] == ':':
          return str(int(string[:1]) + 12) + string[1:5]
      else:
          return str(int(string[:2]) + 12) + string[2:5]

def extract_hour(time_str):
    if time_str[-2:] >= "30":
      if time_str[:2] == "19":
        return 20
      elif time_str[0] == '9':
        return 10
      elif time_str[:2] == '23':
        return 0
      elif time_str[1] == ':':
        return (int(time_str[0]) + 1)
      else:
        return (int(time_str[:2]) + 1)
    else:
      if  time_str[1] == ":":
        return int(time_str[0])
      else:
        return int(time_str[:2])

df["Hour"] = df["Time"].apply(convert)
df['Hour'] = df['Hour'].apply(extract_hour) #Making new column of hours
grouped = df.groupby(["Name", "Hour"]).count()

#Individual
all_hours = range(24)
for name in grouped.index.levels[0]:
  sub_df = grouped.loc[name].reset_index()
  sub_df = sub_df.sort_values(by = "Hour")
  missing_hours = pd.Series(all_hours).isin(sub_df['Hour']).apply(lambda x: not x)
  sub_df = sub_df.append(pd.DataFrame({'Hour': missing_hours[missing_hours].index}))
  sub_df = sub_df.sort_values('Hour').reset_index(drop=True)
  sub_df["Message"] = sub_df["Message"].fillna(0)
  fig = px.line(sub_df, x = 'Hour', y = 'Message', title = name)
  fig.update_layout(xaxis=dict(dtick=1))
  fig.show()

#Whole Group
grouped = df.groupby("Hour")["Message"].size().reset_index()
all_hours = pd.DataFrame({'Hour': range(24)})
grouped = all_hours.merge(grouped, on='Hour', how='left').fillna(0)
fig = px.line(grouped, x = "Hour", y = "Message", title = "All Messages of the group")
fig.update_layout(xaxis=dict(dtick=1))
fig.show()

<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



...

<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



<ipython-input-9-39931954611e>:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [10]:
def awc(x, y):
  for name in list(x.keys()):
    x[name] = (x[name] / y[name])
  return x

y = dict(df["Name"].value_counts())

new_df = df[df.Message != "<Media omitted>"] #Removing Medai ommitted so that they do not hamper word counts

z = df["Name"].unique() #List of unique names

#Dictionary to count the number of words in each name
x = {}
for name in z:
  x[name] = 0
for i in range(new_df.shape[0]):
  x[df["Name"].iloc[i]] += len(new_df["Message"].iloc[i].split())

out = awc(x, y)

keys = list(x.keys())
values = list(x.values())
sorted_value_index = np.argsort(values)
x = {keys[i]: values[i] for i in sorted_value_index}
x = dict(reversed(list(x.items())))

print("Average Words per Message\n")
keys = list(x.keys())
for name in keys:
  print(name + ": " + str(x[name]) + "\n")

Average Words per Message

Bhavin Jobanputra: 5.721193247008687

Manas Patil: 4.45489443378119

Parth Popat: 4.415171288743883

Sacchit Wathe: 4.375954198473282

Akhil Padmanabhan: 4.359550561797753

Shubham Bhimani: 4.064037558685446

Suraj Shinde: 4.019398258115598

Aditya Repe: 3.9839662447257385

Jatin Pareek: 3.6859270672829996

Krishit Mehta: 3.678474114441417

Ansh Shah: 3.5212077641984183

Dhruv Chauhan: 3.333053691275168

Gennady Korotkevich: 3.1783729156139464

+91 86378 23118: 1.8333333333333333



In [11]:
df["Day"] = df["Date"].dt.day_name()

grouped = df.groupby(["Day", "Hour"]).count()
cat_dtype = pd.CategoricalDtype(categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
grouped.index.set_levels(grouped.index.levels[0].astype(cat_dtype), level=0, inplace=True)
grouped = grouped.sort_values("Day")
grouped = grouped.reset_index()

fig = px.bar(grouped, x="Hour", y="Message", color="Day", title="Daily Number of message by Hour", color_discrete_sequence=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2'])
fig.show()

fig = px.histogram(grouped, x="Hour", y="Message", color='Day', barmode='group', title = "Better Plot", color_discrete_sequence=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2'])
fig.show()

#Different color options
# fig = px.histogram(grouped, x="Hour", y="Message", color='Day', barmode='group', color_discrete_sequence=['#FF6361', '#58508d', '#bc5090', '#FFA600', '#003f5c', '#7f33a6', '#FFD700'])
# fig = px.histogram(grouped, x="Hour", y="Message", color='Day', barmode='group', color_discrete_sequence=['#f2a154', '#e74c3c', '#c24e3c', '#f7dc6f', '#d1ccc0', '#e67e22', '#f1c40f'])
# fig = px.histogram(grouped, x="Hour", y="Message", color='Day', barmode='group', color_discrete_sequence=['#4B778D', '#4FB99F', '#F9CDAD', '#C6D8D3', '#F6BD60', '#FFE66D', '#F16B6F'])
# fig = px.histogram(grouped, x="Hour", y="Message", color='Day', barmode='group', color_discrete_sequence=['#F7A278', '#F4B4B4', '#A2B8C4', '#D9CAB3', '#F9DAD0', '#A4D4AE', '#D4A4A4'])
# fig = px.histogram(grouped, x="Hour", y="Message", color='Day', barmode='group', color_discrete_sequence=['#9e7a4c', '#6b4e31', '#5f5b3f', '#85815e', '#bd9e39', '#d2b48c', '#cd853f'])
# fig = px.histogram(grouped, x="Hour", y="Message", color='Day', barmode='group', color_discrete_sequence=['#e6194B', '#3cb44b', '#f58231', '#911eb4', '#ffe119', '#42d4f4', '#f032e6'])

<ipython-input-11-b5d36a8a1722>:5: FutureWarning:

inplace is deprecated and will be removed in a future version.



In [20]:
#Accuracy

newdf = df[df.Message != "<Media omitted>"]

X = newdf['Message']
y = newdf['Name']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

clf1 = MultinomialNB()
clf1.fit(X_train_vec, y_train)

X_test_vec = vectorizer.transform(X_test)

y_pred = clf1.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(accuracy))

Accuracy: 0.3505813210680976


In [13]:
# #Only based on Message data

# newdf = df[df.Message != "<Media omitted>"]

# X = newdf['Message']
# y = newdf['Name']

# #Vectorize the text data
# vectorizer = CountVectorizer()
# X_vec = vectorizer.fit_transform(X)

# clf1 = MultinomialNB()
# clf1.fit(X_vec, y)

In [14]:
# #All, Message, hour and day

# newdf = df[df.Message != "<Media omitted>"]

# X = newdf[['Message', 'Day', 'Hour']]
# y = newdf['Name']

# X = pd.get_dummies(X, columns=['Day']) #Convert categorical data to numerical data

# #Vectorize the text data
# vectorizer = CountVectorizer()
# X_vec = vectorizer.fit_transform(X['Message'])

# X_vec_df = pd.DataFrame(X_vec.toarray()) #Create a DataFrame of the vectorized messages

# #Add column names to the feature matrix
# X_new = pd.concat([X_vec_df, X[['Hour', 'Day_Friday', 'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'Day_Wednesday']].reset_index(drop=True)], axis=1)
# X_new.columns = X_new.columns.astype(str)

# #Train the model
# clf3 = MultinomialNB()
# clf3.fit(X_new, y)

In [21]:
# def predict_sender3(msg, hour, day):
#     msg_vec = vectorizer.transform([msg])

#     hour_day = pd.get_dummies(pd.DataFrame({'Hour': [hour], 'Day': [day]}), columns=['Hour', 'Day'])

#     #Add the Hour and Day columns in the feature matrix and reorder the columns to match the order in X_new
#     msg_vec = pd.concat([pd.DataFrame(msg_vec.toarray()), hour_day], axis=1)
#     msg_vec = msg_vec.reindex(columns=X_new.columns, fill_value=0)
#     msg_vec.columns = msg_vec.columns.astype(str)

#     sender = clf3.predict(msg_vec)[0]

#     #Predict the probabilities of all unique names
#     probas = clf3.predict_proba(msg_vec)[0]
#     prob_dict = dict(zip(clf3.classes_, probas))

#     sorted_probas = sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)
#     print("Predicted Sender: {}\n".format(sender))
#     for name, proba in sorted_probas:
#         print("- {}: {:.2f}%".format(name, proba*100))

def predict_sender1(msg):
    msg_vec = vectorizer.transform([msg])

    sender = clf1.predict(msg_vec)[0]

    # Predict the probabilities of all unique names
    probas = clf1.predict_proba(msg_vec)[0]
    prob_dict = dict(zip(clf1.classes_, probas))

    sorted_probas = sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)
    print("Text: {}\n".format(msg))
    print("Predicted Sender: {}\n".format(sender))
    for name, proba in sorted_probas:
        print("- {}: {:.2f}%".format(name, proba*100))


msg = "For survival hanging out together is necessary innit."
# hour = 18
# day = "Thursday"
# print("Message: {}\nHour: {}\nDay: {}\n\n".format(msg, hour, day))
# print("Taking into consideration hour, day and message")
# predict_sender3(msg, hour, day)
#print("\n\nTaking into consideration only message")
predict_sender1(msg)

Text: For survival hanging out together is necessary innit.

Predicted Sender: Manas Patil

- Manas Patil: 46.05%
- Akhil Padmanabhan: 23.18%
- Jatin Pareek: 10.07%
- Bhavin Jobanputra: 6.80%
- Suraj Shinde: 6.44%
- Ansh Shah: 2.28%
- Sacchit Wathe: 2.19%
- Gennady Korotkevich: 1.78%
- Shubham Bhimani: 0.93%
- Aditya Repe: 0.14%
- Parth Popat: 0.11%
- Dhruv Chauhan: 0.03%
- Krishit Mehta: 0.01%
- +91 86378 23118: 0.00%


In [24]:
#Accuracy

newdf = df[df.Message != "<Media omitted>"]
newdf = newdf[newdf.Message != " <Media omitted>"]

messages = newdf['Message']
senders = newdf['Name']

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(messages)

X_train, X_test, y_train, y_test = train_test_split(X, senders, test_size=0.2, random_state=42)

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3414026836353576


In [36]:
def predict_sender_probability(text):
    text_vectorized = vectorizer.transform([text])
    probabilities = classifier.predict_proba(text_vectorized)[0]
    sender_probabilities = dict(zip(classifier.classes_, probabilities))
    sender_probabilities = sorted(sender_probabilities.items(), key=lambda x: x[1], reverse = True)

    print("Text: {}\n".format(text))
    print("Predicted Sender: {}\n".format(sender_probabilities[0][0]))
    for key, value in sender_probabilities:
      print("- {}: {:.2f}%".format(key, value*100))

input_text = "For survival hanging out together is necessary innit."
predict_sender_probability(input_text)

Text: For survival hanging out together is necessary innit.

Predicted Sender: Jatin Pareek

- Jatin Pareek: 31.17%
- Manas Patil: 21.00%
- Shubham Bhimani: 12.00%
- Suraj Shinde: 10.39%
- Ansh Shah: 6.09%
- Sacchit Wathe: 5.85%
- Akhil Padmanabhan: 4.62%
- Bhavin Jobanputra: 3.49%
- Dhruv Chauhan: 2.00%
- Gennady Korotkevich: 2.00%
- Parth Popat: 1.00%
- Aditya Repe: 0.40%
- Krishit Mehta: 0.00%
